In [ ]:
!pip install -e .. > /dev/null 2>&1

In [ ]:
import os
import json
import astrohack

import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [20, 20]

In [ ]:
def build_folder_structure(paths):
    for path in paths:
        if not os.path.exists(path):
            os.makedirs(path) 

datafolder = 'data'
resultsfolder = 'results'

# Build folder structure & download 
build_folder_structure(['data', 'results'])

toolviper.utils.data.download('J1924-2914.ms.calibrated.split.SPW3', folder=datafolder, unpack=True)
toolviper.utils.data.download(file='extract_holog_verification.json')
toolviper.utils.data.download(file='holog_numerical_verification.json')

In [ ]:
from toolviper.dask.client.client import local_client

log_params = {'log_level':'DEBUG'}

client = local_client(cores=2, memory_limit='8GB', log_params=log_params)

In [ ]:
def test_holog_obs_dictionary(holog_obs_dict):
    import json
    

    with open(".holog_obs_dict.json") as json_file:
        holog_obj = json.load(json_file)    
    
    assert json.loads(holog_obj) == holog_obs_dict, "Error: holog_obs_descrition dictionary has changes unexpectedly."
    
def test_holog_diagnostics(json_data, tolerance=1e-7):
    import json
    
    with open("holog_numerical_verification.json") as file:
        reference_dict = json.load(file)
        
    cell_size = reference_dict["alma"]['cell_size'][1]
    grid_size = float(reference_dict["alma"]['grid_size'][1])
    
    json_data['cell_size'] = np.abs(float(json_data['cell_size']))
    
    cell_size = np.abs(float(cell_size))
    
    assert relative_difference(json_data['cell_size'], cell_size) < tolerance, "Unexpected change in cell_size occured."
    assert relative_difference(np.sqrt(int(json_data['n_pix'])), grid_size) < tolerance, "Unexpected change in grid_size occured."

    
def test_center_pixel(file, antenna, ddi, reference_center_pixels, number_of_digits=7):
    from astrohack.dio import open_image
    
    mds = open_image(file)[antenna][ddi]
    
    aperture_shape = mds.APERTURE.values.shape[-2], mds.APERTURE.values.shape[-1]
    beam_shape = mds.BEAM.values.shape[-2], mds.BEAM.values.shape[-1]  
    
    aperture_center_pixels = np.squeeze(mds.APERTURE.values[..., aperture_shape[0]//2, aperture_shape[1]//2])
    beam_center_pixels = np.squeeze(mds.BEAM.values[..., beam_shape[0]//2, beam_shape[1]//2])
    
    aperture_ref = list(map(complex, reference_center_pixels['aperture']))
    beam_ref = list(map(complex, reference_center_pixels['beam']))
    
    for i in range(len(aperture_ref)):
        assert relative_difference(
            aperture_ref[i].real, 
            aperture_center_pixels[i].real
        ) < 1e-6, "There has been a shift in aperture center pixel value(s)"
        
        assert relative_difference(
            beam_ref[i].real, 
            beam_center_pixels[i].real
        ) < 1e-6, "There has been a shift in beam center pixel value(s)"
                
        assert relative_difference(
            aperture_ref[i].imag, 
            aperture_center_pixels[i].imag
        ) < 1e-6, "There has been a shift in aperture center pixel value(s)"
        assert relative_difference(
            beam_ref[i].imag, 
            beam_center_pixels[i].imag
        ) < 1e-6, "There has been a shift in beam center pixel value(s)"
        
    
def get_center_pixel(file, antenna, ddi):
    from astrohack.dio import open_image
    
    mds = open_image(file)[antenna][ddi]
    
    aperture_shape = mds.APERTURE.values.shape[-2], mds.APERTURE.values.shape[-1]
    beam_shape = mds.BEAM.values.shape[-2], mds.BEAM.values.shape[-1]    
    
    aperture_center_pixels = mds.APERTURE.values[..., aperture_shape[0]//2, aperture_shape[1]//2]
    beam_center_pixels = mds.BEAM.values[..., beam_shape[0]//2, beam_shape[1]//2]
    
    return np.squeeze(aperture_center_pixels), np.squeeze(beam_center_pixels)

def compare_float_values(result, reference, decimals):
    result_list = list(map(str, str(result)))
    reference_list = list(map(str, str(reference)))
    
    return result_list[:decimals] == reference_list[:decimals]

def relative_difference(result, expected):
    return 2*np.abs(result - expected)/(abs(result) + abs(expected))

In [ ]:
from astrohack.extract_pointing import extract_pointing

alma_ms = "data/J1924-2914.ms.calibrated.split.SPW3"

extract_pointing(
    ms_name=alma_ms,
    point_name='results/alma.split.point.zarr',
    parallel=False,
    overwrite=True
)

In [ ]:
import numpy as np

from astrohack.extract_holog import extract_holog

with open("extract_holog_verification.json") as file:
    json_dict = json.load(file)
    
holog_obs_dict = json_dict["alma"]

alma_ms = "data/J1924-2914.ms.calibrated.split.SPW3"
alma_holog = 'results/alma.split.holog.zarr'

extract_holog(
    ms_name=alma_ms,
    holog_name=alma_holog,
    point_name='results/alma.split.point.zarr',
    data_column='DATA',
    parallel=False,
    overwrite=True
)

test_holog_obs_dictionary(holog_obs_dict)

with open('results/alma.split.holog.zarr/.holog_attr') as attr_file:
    holog_attr = json.load(attr_file)
    
test_holog_diagnostics(
    json_data=holog_attr,
    tolerance=2e-5
)

In [ ]:
import numpy as np
from astrohack import holog

with open("holog_numerical_verification.json") as file:
    reference_dict = json.load(file)

grid_interpolation_mode = 'linear' #'nearest' 'linear' 'cubic'
chan_average = True
scan_average = True

holog(
    holog_name=alma_holog, 
    padding_factor=50, 
    grid_interpolation_mode=grid_interpolation_mode,
    chan_average = chan_average,
    scan_average = scan_average,
    overwrite=True,
    phase_fit_engine="perturbations",
    apply_mask=True,
    to_stokes=True,
    parallel=True
)
    
test_center_pixel(
    file="results/alma.split.image.zarr", 
    antenna="ant_DV13", 
    ddi="ddi_0", 
    reference_center_pixels=reference_dict["alma"]['pixels']
)

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import scipy
import matplotlib.patches as patches

from scipy import constants

from astrohack.dio import open_image

alma_image = resultsfolder+'/alma.split.image.zarr'

plt.close('all')

chan = 0

ds = open_image("results/alma.split.image.zarr")['ant_DV13']['ddi_0']

wavelength = scipy.constants.speed_of_light/ds.chan.values[chan]
l = ds.l.values
m = ds.m.values
u = ds.u.values*wavelength
v = ds.v.values*wavelength

print(u.min(),u.max())

plt.figure()
plt.imshow(np.abs(ds.BEAM[0,chan,0,:,:]),extent=[l.min(), l.max(), m.min(), m.max()])
plt.colorbar()
plt.show()

circle = patches.Circle((0,0), 12.5, fill=False, color='white', alpha=0.7, linewidth=2)

fig, ax =plt.subplots()

plt.imshow(np.abs(ds.APERTURE[0,chan,0,:,:]),extent=[u.min(), u.max(), v.min(), v.max()])
plt.colorbar()
plt.show()

ds

In [ ]:
from astrohack.panel import panel

panel_model = 'rigid'

alma_panel = panel(
    image_name=alma_image, 
    panel_model=panel_model,
    parallel=True,
    overwrite=True
)

In [ ]:
import numpy as np
from astrohack.dio import open_panel

def relative_difference(mean, expected):  
    return 2*np.abs(mean - expected)/(abs(mean) + abs(expected))

def verify_panel_positions(
    panel_list=['3-11', '5-31', '7-52', '11-62'], 
    reference_position = np.array([-2.16823971, -0.94590908,  0.84834425, 0.76463105]),
    antenna='ant_DV13',
    ddi='ddi_0'
):
    
    M_TO_MILS = 39370.1
    
    panel_mds = open_panel('results/alma.split.panel.zarr')
    
    panel_position = np.mean(panel_mds[antenna][ddi].sel(labels=panel_list).PANEL_SCREWS.values*M_TO_MILS, axis=1)

    relative_position = relative_difference(panel_position, reference_position)
    
    if np.any(relative_position > 1e-6): 
        print("There were changes!")
        print(relative_position)    

In [ ]:
verify_panel_positions(antenna='ant_DV13', ddi='ddi_0')